**Devènes Florian & Robyr Eloi**

# Projet : analyse séquentielle de data 

## Analyse de la série temporelle des utilisateurs connectés à l'environnement steam

In [ ]:
# Import libraries
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltC


ModuleNotFoundError: No module named 'pandas'

In [6]:
# Cut data when until 3 consecutive lines don't have NaN in 'Users' and 'Average Users'

def cut_data(df):
    count = 0
    cut_index = len(df)  # Default to full length if no cut is found

    for i in range(len(df)-1, -1, -1):
        if pd.notna(df.loc[i, 'Users']) and pd.notna(df.loc[i, 'Average Users']):
            count += 1
            if count == 3:
                cut_index = i
                break
        else:
            count = 0

    return df.iloc[cut_index:]

**Data pull**

In [7]:
data_dir = 'Data'
data_dict = {}

for file in os.listdir(data_dir):
    if file.endswith('.csv'):
        # Exemple de nom de fichier : steamdb_chart_data_10nov25.csv
        match = re.search(r"steamdb_chart_data_(.*?)\.csv", file)
        if match:
            pulled_data_date = match.group(1)  # extrait "xxx"
            df = pd.read_csv(os.path.join(data_dir, file),sep=';')
            df["DateTime"] = pd.to_datetime(df["DateTime"])
            df["Extraction date"] = pulled_data_date
            data_dict[pulled_data_date] = df
            print('Fichier chargé :', file, 'avec la date extraite :', pulled_data_date)
        else:
            print(f"Nom de fichier ignoré : {file}")




NameError: name 'pd' is not defined

In [ ]:
# ensure sorted
df = df.sort_values('DateTime')

# compute the time differences
df["dt"] = df["DateTime"].diff()

# find where frequency changes (i.e., dt != previous dt)
df["dt_change"] = df["dt"] != df["dt"].shift()

# extract only change points (skip the first NaN)
changes = df.loc[df["dt_change"] & df["dt"].notna(), ["DateTime", "dt"]]

# add old and new frequencies
changes["old_freq"] = changes["dt"].shift()
changes["new_freq"] = changes["dt"]

# clean up
changes = changes[["DateTime", "old_freq", "new_freq"]].dropna()
print(changes)


                DateTime        old_freq        new_freq
7943 2025-10-11 01:00:00 1 days 00:00:00 0 days 01:00:00
8506 2025-11-03 11:10:00 0 days 01:00:00 0 days 00:10:00


In [ ]:
import pandas as pd
import numpy as np

# Conversion DateTime
df["DateTime"] = pd.to_datetime(df["DateTime"])

# On garde uniquement Users
df = df[["DateTime", "Users"]]

# On s’assure que les données sont triées
df = df.sort_values("DateTime").reset_index(drop=True)

# Daily data by selecting one value per day (or resampling)
df_daily = df.resample("1D", on="DateTime").mean().dropna()

# Training sequences (30 jours → 30*24 heures)
WINDOW = 30
UPSCALE = 24


In [ ]:
import pandas as pd
import numpy as np

df["DateTime"] = pd.to_datetime(df["DateTime"])
df = df[["DateTime", "Users"]].sort_values("DateTime")

# Daily sampling
df_daily = df.resample("1D", on="DateTime").mean().dropna()

# Hourly interpolation (pour construire Y)
df_hourly = df.set_index("DateTime").resample("1H").interpolate()


C:\Users\xenio\AppData\Local\Temp\ipykernel_17072\82665578.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.set_index("DateTime").resample("1H").interpolate()


In [ ]:
WINDOW = 30
UPSCALE = 24

X, Y = [], []

daily_values = df_daily["Users"].values
hourly_values = df_hourly["Users"].values

for i in range(len(df_daily) - WINDOW):
    x_seq = daily_values[i : i + WINDOW]          # (30,)
    y_seq = hourly_values[i*24 : (i + WINDOW)*24] # (720,)
    
    X.append(x_seq)
    Y.append(y_seq)

X = np.array(X).reshape(-1, WINDOW, 1)
Y = np.array(Y).reshape(-1, WINDOW*24, 1)

print("X:", X.shape, "Y:", Y.shape)


X: (6733, 30, 1) Y: (6733, 720, 1)


In [ ]:
from tensorflow.keras import layers, models

def build_model(hidden=64):
    encoder_inputs = layers.Input(shape=(WINDOW, 1))
    encoder_outputs, state_h, state_c = layers.LSTM(hidden, return_state=True)(encoder_inputs)

    # Decoder: génère 720 timestamps
    decoder_inputs = layers.Input(shape=(WINDOW*24, 1))
    decoder_lstm = layers.LSTM(hidden, return_sequences=True)
    decoder_outputs = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])

    outputs = layers.Dense(1)(decoder_outputs)

    model = models.Model([encoder_inputs, decoder_inputs], outputs)
    model.compile(optimizer="adam", loss="mse")
    return model

model = build_model()
model.summary()


ImportError: Traceback (most recent call last):
  File "c:\Users\xenio\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Une routine d’initialisation d’une bibliothèque de liens dynamiques (DLL) a échoué.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
decoder_zeros = np.zeros((len(X), WINDOW*24, 1))

model.fit([X, decoder_zeros], Y, epochs=20, batch_size=16)


In [ ]:
x_last = df_daily["Users"].values[-WINDOW:]
x_last = x_last.reshape(1, WINDOW, 1)

decoder_zero = np.zeros((1, WINDOW*24, 1))
generated = model.predict([x_last, decoder_zero])[0].reshape(-1)


In [ ]:
start = df_daily.index[-WINDOW]
generated_index = pd.date_range(start, periods=WINDOW*24, freq="1H")

df_gen = pd.DataFrame({
    "DateTime": generated_index,
    "Generated_Users": generated
})


In [ ]:
df_gen_daily = (
    df_gen.groupby(df_gen["DateTime"].dt.date)
          .apply(lambda g: g.sample(1))
          .reset_index(drop=True)
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(df_gen_daily["DateTime"], df_gen_daily["Generated_Users"], marker="o")
plt.title("Données générées (1 valeur par jour, heure aléatoire)")
plt.xlabel("Date")
plt.ylabel("Generated Users")
plt.grid(True)
plt.show()
